In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle

In [51]:
codebook_xl = pd.ExcelFile('Datasets/codebook_publicv4.xlsx')
codebook_df = codebook_xl.parse('Sheet1', header=3, usecols=[0,1,3,4,5,6,7,8,9],
                                nrows =758, index_col = 0, skiprows =[1])
codebook_df.columns = ['type','variable_description','feature','include', 'output_input','data_type',
                       'response_values', 'response_labels']
codebook_df.include = codebook_df.include.replace('X',True).fillna(False)
codebook_df.feature = codebook_df.feature.replace('X',True).fillna(False)
codebook_df.index.names = ['Variables']

In [35]:
variables = codebook_df.index.values
descriptions = codebook_df.variable_description.values
var_map = {var:None for var in variables}
var_descriptions = dict(zip(variables,descriptions))

In [36]:
labels = codebook_df.response_labels
label_strings = [l.replace('\n','; ') if type(l) is str else l for l in labels ]
var_labels = dict(zip(labels.index,label_strings))

In [37]:
#Create mask for response_values that are strings in the dataframe
is_string = codebook_df['response_values'].apply(isinstance,args=(str,))

#response_values that are strings in the dataframe
response_values = codebook_df[is_string]['response_values']

#Corresponding label_set for the response_values 
label_sets = codebook_df[is_string]['response_labels']

#Variables with string response_values in the dataframe
str_response_vars = codebook_df[is_string]['response_labels'].index

In [38]:
#Split response_values strings into a list of values and create list of value sets for each row
val_list = [entry.split('\n') for entry in response_values.values]

#Split response_labels strings into a list labels and create list of label sets for each row
lab_list = [entry.split('\n') for entry in label_sets.values]

#Create a list of dictionaries with response variables as keys and the corressponding labels as values
label_map = [dict(zip(vals,labels)) for vals,labels in zip(val_list,lab_list)]

#Assign the dictionary of response values and labels to the correspnign variable
for var,lab in zip(str_response_vars,label_map):
    var_map[var] = lab

In [39]:
#Create mask for null response_values
null_values = codebook_df['response_values'].isnull()

#Create mask for null response_labels
null_labels = codebook_df['response_labels'].isnull()

#Create mask for null numeric variables
is_numeric = codebook_df['type'] =='Numeric'

#Create mask for null character variables
is_character = codebook_df['type'] =='Character'

In [40]:
numeric_labels = codebook_df[is_numeric & np.invert(null_labels) & null_values].response_labels
non_null_labels = codebook_df[is_numeric & np.invert(null_labels) & null_values].index

for var,label in zip(non_null_labels,numeric_labels):
    var_map[var] = {'Numeric':label}

In [41]:
null_character = codebook_df.loc[is_character & null_values].index
label_str = codebook_df.loc[is_character & null_values]['response_labels'].values
label_list = label_str[0].split('\n')
var_map[null_character.values[0]] = dict(zip(label_list,label_list))

In [42]:
string_responses = codebook_df[codebook_df['type'] =='Character'].response_values
string_responses.index

Index(['METROMICRO', 'UATYP10', 'CLIMATE_REGION_PUB', 'IECC_CLIMATE_PUB'], dtype='object', name='Variables')

In [43]:
var_map['IECC_CLIMATE_PUB']

{'1A-2A': 'IECC climate zones 1A-2A',
 '2B': 'IECC climate zone 2B',
 '3A': 'IECC climate zone 3A',
 '3B-4B': 'IECC climate zones 3B-4B',
 '3C': 'IECC climate zone 3C',
 '4A': 'IECC climate zone 4A',
 '4C': 'IECC climate zone 4C',
 '5A': 'IECC climate zone 5A',
 '5B-5C': 'IECC climate zones 5B-5C',
 '6A-6B': 'IECC climate zones 6A-6B',
 '7A-7B-7AK-8AK': 'IECC climate zones 7A-7B-7AK-8AK'}

In [44]:
var_map['METROMICRO'] = {1: 'Housing unit in Census Metropolitan Statistical Area',
                         2: 'Housing unit in Census Micropolitan Statistical Area',
                         3: 'Housing unit in neither'}
var_map['UATYP10'] = {1: 'Urban Area', 3: 'Urban Cluster', 3: 'Rural'}
var_map['CLIMATE_REGION_PUB'] = {1: 'Cold/Very Cold', 2: 'Hot-Dry/Mixed-Dry', 3: 'Hot-Humid', 4: 'Mixed-Humid',
                                 5: 'Marine'}
var_map['IECC_CLIMATE_PUB'] = {1: 'IECC climate zones 1A-2A', 2: 'IECC climate zone 2B',
                               3: 'IECC climate zone 3A', 4: 'IECC climate zones 3B-4B',
                               5: 'IECC climate zone 3C', 6: 'IECC climate zone 4A',
                               7: 'IECC climate zone 4C', 8: 'IECC climate zone 5A',
                               9: 'IECC climate zones 5B-5C', 10: 'IECC climate zones 6A-6B',
                               11: 'IECC climate zones 7A-7B-7AK-8AK'}


In [52]:
data_info = codebook_df[['feature','include','data_type']]
data_info.to_pickle('Datasets/data_info.pickle')

In [46]:
with open('Datasets/variable_map.pickle', 'wb') as handle:
    pickle.dump(var_map, handle)

In [47]:
with open('Datasets/variable_descriptions.pickle', 'wb') as handle:
    pickle.dump(var_descriptions, handle)

In [48]:
with open('Datasets/variable_labels.pickle', 'wb') as handle:
    pickle.dump(var_labels, handle)